# Meridian Marketing Mix Modeling (MMM) Analysis

This notebook demonstrates how to use Google's Meridian framework for Marketing Mix Modeling analysis.

## Overview

Marketing Mix Modeling (MMM) is a statistical analysis technique that helps businesses understand the impact of various marketing activities on sales and other key performance indicators. Google's Meridian framework provides a comprehensive solution for MMM analysis using Bayesian methods.

## Key Features

- **Bayesian Causal Inference**: Advanced statistical methods to measure marketing impact
- **Media Attribution**: Quantify the contribution of each media channel
- **Budget Optimization**: Recommend optimal budget allocation across channels
- **Saturation Effects**: Model diminishing returns of media spend
- **Adstock Effects**: Account for delayed impact of advertising

## Prerequisites

Before running this notebook, ensure you have:

1. Installed the required dependencies (see requirements.txt)
2. Installed Meridian from GitHub: `pip install git+https://github.com/google/meridian.git`
3. Activated your virtual environment

## Table of Contents

1. [Setup and Imports](#setup)
2. [Data Preparation](#data-prep)
3. [Model Training](#model-training)
4. [Results Analysis](#results)
5. [Budget Optimization](#budget-opt)
6. [Visualization](#visualization)
7. [Report Generation](#report)


## 1. Setup and Imports {#setup}

First, let's import all necessary libraries and set up the environment.


In [1]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import sys

# Set up plotting
%matplotlib inline
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Add project root to path
sys.path.insert(0, os.path.abspath('..'))

# Import our custom modules
from src.data_preprocess.meridian.data_preprocess_utils import (
    create_sample_mmm_data,
    create_mmm_data_structure,
    validate_mmm_data,
    preprocess_mmm_data
)
from src.model.meridian.meridian_model import MeridianMMMModel
from src.model.meridian.mmm_analysis_poc import (
    load_mmm_data,
    preprocess_mmm_data_for_modeling,
    fit_mmm_model,
    analyze_mmm_results,
    optimize_media_budget,
    save_mmm_results
)
from src.model.meridian.visualization import (
    plot_media_attribution,
    plot_model_performance,
    plot_budget_optimization,
    plot_media_response_curves,
    plot_media_contribution_timeseries,
    plot_media_target_curves,
    plot_prior_posterior,
    generate_mmm_html_report
)

print("✅ All imports successful!")
print(f"📊 NumPy version: {np.__version__}")
print(f"🐼 Pandas version: {pd.__version__}")
print(f"📈 Matplotlib version: {plt.matplotlib.__version__}")


ModuleNotFoundError: No module named 'src.data_preprocess.meridian.data_preprocess_utils'

## 2. Data Preparation {#data-prep}

In this section, we'll prepare the data for MMM analysis. We'll use sample data that mimics real-world marketing mix data.


In [ ]:
# Configuration
N_WEEKS = 104  # 2 years of weekly data
N_GEOS = 5     # 5 geographic regions
MEDIA_CHANNELS = ['TV', 'Digital', 'Radio', 'Print', 'Outdoor']
CONTROL_VARS = ['Price', 'Promotion', 'Seasonality']

print(f"📊 Generating sample MMM data...")
print(f"   - Time period: {N_WEEKS} weeks")
print(f"   - Geographic regions: {N_GEOS}")
print(f"   - Media channels: {MEDIA_CHANNELS}")
print(f"   - Control variables: {CONTROL_VARS}")

# Generate sample data
sample_data = create_sample_mmm_data(
    n_weeks=N_WEEKS,
    n_geos=N_GEOS,
    media_channels=MEDIA_CHANNELS,
    control_vars=CONTROL_VARS
)

print("\n✅ Sample data generated successfully!")
print(f"   - Media data shape: {sample_data['media_data'].shape}")
print(f"   - Target data shape: {sample_data['target_data'].shape}")
print(f"   - Control data shape: {sample_data['control_data'].shape}")


In [ ]:
# Display sample data
print("📋 Sample Media Data:")
display(sample_data['media_data'].head())

print("\n📋 Sample Target Data:")
display(sample_data['target_data'].head())

print("\n📋 Sample Control Data:")
display(sample_data['control_data'].head())


In [ ]:
# Convert to Meridian format
data_dict = create_mmm_data_structure(
    sample_data['media_data'],
    sample_data['target_data'],
    sample_data['control_data']
)

print("🔄 Data converted to Meridian format:")
print(f"   - Media array shape: {data_dict['media'].shape}")
print(f"   - Target array shape: {data_dict['target'].shape}")
print(f"   - Control array shape: {data_dict['control'].shape}")
print(f"   - Media channels: {data_dict['media_channels']}")
print(f"   - Control variables: {data_dict['control_vars']}")

# Validate data
if validate_mmm_data(data_dict):
    print("\n✅ Data validation passed!")
else:
    print("\n⚠️ Data validation failed, but proceeding...")


In [ ]:
# Preprocess data for modeling
processed_data = preprocess_mmm_data_for_modeling(
    data_dict,
    log_transform=True,
    standardize=False  # Keep original scale for interpretability
)

print("🔧 Data preprocessing completed:")
print(f"   - Log transformation: Applied")
print(f"   - Standardization: Not applied")
print(f"   - Final media shape: {processed_data['media'].shape}")
print(f"   - Final target shape: {processed_data['target'].shape}")


## 3. Model Training {#model-training}

Now we'll train the Meridian MMM model using Bayesian inference.


In [ ]:
# Model configuration
NUM_SAMPLES = 1000  # Number of MCMC samples
NUM_WARMUP = 500    # Number of warmup samples
NUM_CHAINS = 2      # Number of MCMC chains
RANDOM_SEED = 42    # Random seed for reproducibility

print(f"🤖 Training Meridian MMM model...")
print(f"   - MCMC samples: {NUM_SAMPLES}")
print(f"   - Warmup samples: {NUM_WARMUP}")
print(f"   - Chains: {NUM_CHAINS}")
print(f"   - Random seed: {RANDOM_SEED}")

# Fit the model
model = fit_mmm_model(
    data_dict=processed_data,
    num_samples=NUM_SAMPLES,
    num_warmup=NUM_WARMUP,
    num_chains=NUM_CHAINS,
    random_seed=RANDOM_SEED
)

print("\n✅ Model training completed!")
print(f"   - Model fitted: {model.fitted}")
print(f"   - MCMC samples available: {model.mcmc_samples is not None}")
print(f"   - Trace available: {model.trace is not None}")


## 4. Quick Start - Complete Analysis

For a quick demonstration, let's run the complete MMM analysis pipeline:


In [ ]:
# Run complete MMM analysis
from src.model.meridian.mmm_analysis_poc import run_complete_mmm_analysis

print("🚀 Running complete MMM analysis pipeline...")

# Run with optimized parameters for faster execution
results = run_complete_mmm_analysis(
    use_sample_data=True,
    n_weeks=52,  # 1 year of data
    n_geos=3,    # 3 geographic regions
    num_samples=500,  # Reduced for faster execution
    num_warmup=250,
    num_chains=2,
    total_budget=1000000,  # $1M budget
    output_dir="data/meridian"
)

print("\n🎉 Complete analysis finished!")
print(f"Model Performance - R²: {results['analysis']['performance']['r_squared']:.3f}")
print(f"Media Channels: {results['media_channels']}")
print(f"Control Variables: {results['control_vars']}")


## 5. Using Real Data

To use your own data, follow these steps:

1. **Prepare your data** in CSV format with columns:
   - `date`: Date column
   - `geo`: Geographic identifier
   - `{channel}_spend`: Media spend for each channel
   - `sales`: Target variable
   - Control variables (e.g., `price`, `promotion`)

2. **Load your data**:
   ```python
   results = run_complete_mmm_analysis(
       data_path="path/to/your/data.csv",
       use_sample_data=False,
       num_samples=2000,  # Increase for better results
       num_warmup=1000,
       total_budget=your_budget
   )
   ```

3. **Customize media channels** by modifying the data preprocessing utilities.

## 6. Advanced Features

The Meridian framework supports advanced features like:

- **Geo-level analysis**: Analyze performance across different geographic regions
- **Seasonality modeling**: Capture seasonal patterns in your data
- **Competitive effects**: Model competitor activities
- **Cross-media effects**: Account for interactions between media channels

## 7. Next Steps

1. **Install Meridian**: `pip install git+https://github.com/google/meridian.git`
2. **Run this notebook** to see the analysis in action
3. **Adapt to your data** by modifying the data loading functions
4. **Explore advanced features** in the Meridian documentation

---

*This notebook demonstrates the integration of Google's Meridian MMM framework with our custom analysis pipeline.*
